In [1]:
import os
import urllib
import traceback
import time
import sys
import numpy as np
import cv2
from utils import preproc as preprocess
from rknn.api import RKNN
import time
from utils import mkdir, multiclass_nms, demo_postprocess, vis
from config import Config


In [2]:
cfg = Config()
ONNX_MODEL = '/home/white/PycharmProjects/YOLOX-main/tools/yolox.onnx'
RKNN_MODEL = 'yolox.rknn'
input_shape = tuple(map(int, "640,640".split(',')))

In [3]:
rknn = RKNN()
rknn.config(mean_values=[0.485,0.456,0.406], std_values=[0.229,0.224,0.225])


W config: Generally, when the model input is an image, the range of mean_values/std_values should be in [0, 255], but the value you filled is too small,
          please check carefully!


0

In [4]:
ret = rknn.load_onnx(model=ONNX_MODEL)


In [5]:
ret = rknn.build(do_quantization=False)


Analysing : 100%|███████████████████████████████████████████████| 279/279 [00:00<00:00, 4468.53it/s]


In [6]:
ret = rknn.export_rknn(RKNN_MODEL)

In [7]:
origin_img = cv2.imread(cfg.image_path)

In [8]:
# mean = (0.485, 0.456, 0.406)
# std = (0.229, 0.224, 0.225)

In [9]:
img, ratio = preprocess(origin_img, input_shape,mean=None,std=None)

In [10]:
ret = rknn.init_runtime()

W init_runtime: target is None, use simulator!


In [11]:
outputs = rknn.inference(inputs=[img])

In [12]:
outputs[0].squeeze(axis=3).shape

(1, 8400, 12)

In [13]:
predictions = demo_postprocess(outputs[0].squeeze(axis=3), input_shape)[0]

[array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        ...,
        [77, 79],
        [78, 79],
        [79, 79]]]), array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        ...,
        [37, 39],
        [38, 39],
        [39, 39]]]), array([[[ 0,  0],
        [ 1,  0],
        [ 2,  0],
        [ 3,  0],
        [ 4,  0],
        [ 5,  0],
        [ 6,  0],
        [ 7,  0],
        [ 8,  0],
        [ 9,  0],
        [10,  0],
        [11,  0],
        [12,  0],
        [13,  0],
        [14,  0],
        [15,  0],
        [16,  0],
        [17,  0],
        [18,  0],
        [19,  0],
        [ 0,  1],
        [ 1,  1],
        [ 2,  1],
        [ 3,  1],
        [ 4,  1],
        [ 5,  1],
        [ 6,  1],
        [ 7,  1],
        [ 8,  1],
        [ 9,  1],
        [10,  1],
        [11,  1],
        [12,  1],
        [13,  1],
        [14,  1],
        [15,  1],
        [16,  1],
        [17,  1],
        [18,  1],
        [19,  1],
        [ 0,  2],
        [ 1, 

In [14]:
predictions[2]

array([2.7984375e+01, 9.4218750e+00, 5.4403786e+01, 2.0388855e+01,
       3.0212402e-03, 3.0212402e-03, 3.0212402e-03, 1.0894775e-02,
       5.0354004e-03, 9.2895508e-02, 3.3447266e-01, 1.0437012e-02],
      dtype=float32)

In [15]:
boxes = predictions[:, :4]
scores = predictions[:, 4:5] * predictions[:, 5:]

In [16]:
boxes_xyxy = np.ones_like(boxes)

In [17]:
boxes[:6]

array([[ 6.5429688, 12.8828125, 17.87925  , 37.61087  ],
       [22.375    , 11.484375 , 43.544243 , 25.874804 ],
       [27.984375 ,  9.421875 , 54.403786 , 20.388855 ],
       [33.617188 ,  4.9257812, 39.647484 , 10.950733 ],
       [38.941406 ,  5.2539062, 64.291405 ,  9.164211 ],
       [45.753906 ,  5.1640625, 75.753784 ,  9.032051 ]], dtype=float32)

In [18]:
boxes_xyxy[:, 0] = boxes[:, 0] - boxes[:, 2]/2.
boxes_xyxy[:, 1] = boxes[:, 1] - boxes[:, 3]/2.
boxes_xyxy[:, 2] = boxes[:, 0] + boxes[:, 2]/2.
boxes_xyxy[:, 3] = boxes[:, 1] + boxes[:, 3]/2.
boxes_xyxy /= ratio

In [19]:
boxes_xyxy[:6]


array([[-1.8723875 , -4.627049  , 12.095777  , 24.756445  ],
       [ 0.47099888, -1.1351771 , 34.48994   , 19.079514  ],
       [ 0.6113142 , -0.60355663, 43.114273  , 15.325236  ],
       [10.77613   , -0.42936355, 41.75073   ,  8.125897  ],
       [ 5.3091435 ,  0.5248442 , 55.536808  ,  7.6843843 ],
       [ 6.1539173 ,  0.5062789 , 65.33656   ,  7.562569  ]],
      dtype=float32)

In [20]:
dets = multiclass_nms(boxes_xyxy, scores, nms_thr=0.6, score_thr=0.1)

In [21]:
dets

In [22]:
if dets is not None:
    final_boxes, final_scores, final_cls_inds = dets[:, :4], dets[:, 4], dets[:, 5]
    origin_img = vis(origin_img, final_boxes, final_scores, final_cls_inds,
                     conf=cfg.score_thr, class_names=cfg.COCO_CLASSES)

In [23]:
cv2.imshow("test",origin_img)

In [25]:
cv2.waitKey(0)

48